# Data preparations

In [1]:
!pip install bs4
!pip install pandas
!pip install numpy
!pip install pandas
!pip install scikit-learn
!pip install bokeh
!pip install pyarrow
!pip install fastparquet
!pip install sentence-transformers


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip insta

In [2]:
import requests
from bs4 import BeautifulSoup
import itertools
import pandas as pd
import operator
import numpy as np
import json
import csv
import json
import sqlite3
import itertools
import operator

/Users/nkonovalov/Library/Caches/pypoetry/virtualenvs/hse-homework-0ZnkoyXZ-py3.9/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
def sampleReviews(user_review_ratings):
    
    num_reviews = len(user_review_ratings)
    print(num_reviews)
    if num_reviews < 5:
        return list(range(5))
    indices, reviews_sorted = zip(*sorted(enumerate(user_review_ratings), key=operator.itemgetter(1)))
    if num_reviews % 2 == 0:
        median = int(num_reviews / 2)
    else:
        median = int((num_reviews - 1) / 2)
    x = [0, median-1, median, median+1, num_reviews-1]
    return [indices[i] for i in x]
    

In [4]:
# Load movie data from 'output_with_budget.csv'
movie_csv_file_path = '../datasets/output_with_budget.csv'
movie_df = pd.read_csv(movie_csv_file_path)

# Load movie review data from 'review1.csv'
review_csv_file_path = '../datasets/reviews.csv'
review_df = pd.read_csv(review_csv_file_path)

# Extract IMDb IDs from URLs in 'movie_df'
movie_df['imdb'] = movie_df['imdb'].str.extract(r'tt(\d+)')
movie_df['imdb'] = "/title/tt" + movie_df['imdb'] + "/"

# Merge the two dataframes based on 'imdb'
merged_df = pd.merge(movie_df, review_df, left_on='imdb', right_on='movie_id', how='left')

# Filter out rows where 'user_review_permalink' is NaN
merged_df = merged_df.dropna(subset=['user_review_permalink'])

# Get unique IMDb IDs
unique_imdb_ids = merged_df['imdb'].unique()

# Construct dataframes if needed
review_df = pd.concat([pd.DataFrame({'imdb': [imdb_id], 
                                      'user_review_permalink': [review_row['user_review_permalink']],
                                      'user_review': [review_row['user_review']]})
                      for imdb_id in unique_imdb_ids
                      for _, review_row in merged_df.loc[merged_df['imdb'] == imdb_id].iterrows()],
                     ignore_index=True)

# Replace 'imdb' with the correct column name for IMDb IDs
movie_links = unique_imdb_ids

def getDescs(link):
    # You have data in CSV, so no need for additional scraping
    # Just return the 'description' column value based on the link
    return merged_df.loc[merged_df['imdb'] == link, 'description'].values[0]

# Debug: Print lengths of relevant arrays
print("Length of movie_links:", len(movie_links))
print("Length of review_df['imdb']:", len(review_df['imdb']))

# Replace 'imdb' with the correct column name for IMDb IDs
movie_df = pd.DataFrame({'imdb': movie_links, 'movie_id': unique_imdb_ids,
                         'movie_desc': [getDescs(link) for link in movie_links]})

# Debug: Print length of movie_df['movie_id']
print("Length of movie_df['movie_id']:", len(movie_df['movie_id']))

# Save data to SQLite database
conn = sqlite3.connect("../datasets/movie_search.db")
movie_df.to_sql("movies", conn, if_exists="replace")
review_df.to_sql("movie_reviews", conn, if_exists="replace")

# Save data to Parquet files
movie_df.to_parquet("../datasets/movies.parquet")
review_df.to_parquet("../datasets/reviews.parquet")


Length of movie_links: 6890
Length of review_df['imdb']: 33993
Length of movie_df['movie_id']: 6890


## Film Embeddings

In [5]:
from sentence_transformers import SentenceTransformer

In [6]:
model = SentenceTransformer('stsb-distilbert-base')
model.max_seq_length = 512

In [7]:
embeddings = model.encode(review_df['user_review'], show_progress_bar=True)

Batches:   0%|          | 0/1063 [00:00<?, ?it/s]

In [8]:
def normalize(v):
    norm = np.linalg.norm(v)
    if norm == 0: 
        return v
    return v / norm

In [9]:
# make embeddings part of dataframe, avg by movie title
import numpy as np
avg_embeddings = np.array([sum(embeddings[i:i+5])/5 for i in range(0,len(embeddings),5)])

In [10]:
unit_embeddings = np.array([normalize(v) for v in avg_embeddings])

In [11]:
np.sqrt(np.dot(unit_embeddings[2], unit_embeddings[2]))

0.99999994

In [12]:
embedding_idx_to_movie_id = {idx: iD for idx, iD in enumerate(movie_df.movie_id)}

In [13]:
with open('../datasets/embedding_idx_to_movie_id.json', 'w') as f:
    json.dump(embedding_idx_to_movie_id, f)

In [14]:
np.save('../datasets/movie_embeddings.npy', unit_embeddings)